In [4]:
%%capture
!pip install peft
!pip install accelerate
!pip install bitsandbytes
!pip install faker

In [5]:
import re
import string
import random

import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
from tqdm.notebook import tqdm
from faker import Faker
import pandas as pd
pd.set_option('max_colwidth', None)

In [6]:
MODEL_NAME = "IlyaGusev/saiga_mistral_7b"
DEFAULT_MESSAGE_TEMPLATE = "<s>{role}\n{content}</s>"
DEFAULT_RESPONSE_TEMPLATE = "<s>bot\n"
DEFAULT_SYSTEM_PROMPT = "Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им."

In [7]:
class Conversation:
    def __init__(
        self,
        message_template=DEFAULT_MESSAGE_TEMPLATE,
        system_prompt=DEFAULT_SYSTEM_PROMPT,
        response_template=DEFAULT_RESPONSE_TEMPLATE
    ):
        self.message_template = message_template
        self.response_template = response_template
        self.messages = [{
            "role": "system",
            "content": system_prompt
        }]

    def add_user_message(self, message):
        self.messages.append({
            "role": "user",
            "content": message
        })

    def add_bot_message(self, message):
        self.messages.append({
            "role": "bot",
            "content": message
        })

    def get_prompt(self, tokenizer):
        final_text = ""
        for message in self.messages:
            message_text = self.message_template.format(**message)
            final_text += message_text
        final_text += DEFAULT_RESPONSE_TEMPLATE
        return final_text.strip()


def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(
        **data,
        generation_config=generation_config
    )[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output.strip()

In [8]:
config = PeftConfig.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto"
    )
model = PeftModel.from_pretrained(
    model,
    MODEL_NAME,
    torch_dtype=torch.float16
    )
model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
print(generation_config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/623 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


generation_config.json:   0%|          | 0.00/265 [00:00<?, ?B/s]

GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_new_tokens": 1536,
  "no_repeat_ngram_size": 15,
  "pad_token_id": 0,
  "repetition_penalty": 1.1,
  "temperature": 0.2,
  "top_k": 40,
  "top_p": 0.9
}



In [15]:
def generate_doc_id(length):
  """Generate random string of length {length}
     - will create the document id"""
  letters = string.ascii_lowercase
  return ''.join(random.choice(letters) for _ in range(length))

def generate_name():
  """Generates first and last name."""
  fake = Faker("ru_RU")
  if random.choice([True, False]):
      first_name = fake.first_name_male()
      last_name = fake.last_name_male()
  else:
      first_name = fake.first_name_female()
      last_name = fake.last_name_female()
  return first_name, last_name

def generate_address():
    """Generates a random Russian address with Faker.
       Might be expanded for the sake of diversity."""
    language = "ru_RU"
    fake = Faker(language)
    return fake.address()

In [17]:
address_df = pd.DataFrame()

for i in tqdm(range(1)):

  # Get document ID
  doc_id = generate_doc_id(length=12)
  # Generate PII
  address = generate_address()
  first_name, last_name = generate_name()

  inputs = f"""
      Тебя зовут {first_name} {last_name}. Ты живешь по адресу {address}.
      Тебе необходимо сказать, как тебя зовут, твой полный адрес, а затем написать короткое эссе про твой адрес.
      Пожалуйста, напиши сначала о том, где ты живешь и почему тебе нравится или не нравится там жить.
      Затем расскажи подробности про свое любимое место в мире, и где оно находится.
      НЕ ДАВАЙ НИКАКУЮ ЛИЧНУЮ ИНФОРМАЦИЮ О СЕБЕ КРОМЕ АДРЕСА.
  """.replace("\n ","")

  conversation = Conversation()
  conversation.add_user_message(inputs)
  prompt = conversation.get_prompt(tokenizer)

  output = generate(model, tokenizer, prompt, generation_config)

  # Collect generated PII to label afterwards
  label_dict={"ADDRESS": [address], "NAME": [first_name, last_name]}

  # Collect data and append
  address_df = pd.concat([address_df, pd.DataFrame({"document": [doc_id], "text": [output.replace('\n', ' ')],
                                                    "label_dict": [label_dict]})], axis=0)

address_df.to_csv("pii_address.csv", index=False)
address_df.head()

  0%|          | 0/1 [00:00<?, ?it/s]

,document,text,label_dict
0,zvxpgouvcytz,"Меня зовут Арсений Буров. Я живу в селе Ирбит, улица Механическая, дом 2/2, корпус 3, 143067. Я живу в этом месте уже несколько лет и мне нравится здесь жить. Это красивое село, окруженное лесами и холмами. Здесь всегда много свежего воздуха и тишины, что делает это место идеальным для отдыха и восстановления сил после долгих рабочих дней. Мое любимое место в мире - это горы. Горы представляют собой прекрасные места для осмотра природы, спортивных занятий и просто отдыха. Мне особенно нравятся высокие горы, где можно насладиться видами на окрестности и даже заглянуть в другие страны. Горы также являются источником воды, и я всегда рад посещать реки и озера, которые расположены в их окрестностях. Гора, которую я называю любимой, находится в районе Кавказских гор. Эта гора имеет высоту около 5000 метров и является одной из самых высоких вершин в регионе. На вершине горы расположена горная хижина, где можно остановиться на ночь и насладиться красотой окружающей природы.","{'ADDRESS': ['с. Ирбит, ш. Механическое, д. 2/2 стр. 3, 143067'], 'NAME': ['Арсений', 'Буров']}"
